# Capitulo 05 Arrow Flight Sql

Notebook gerado automaticamente a partir do código fonte python.


In [1]:
# Instalação de pacotes necessários
!pip install "pyarrow[flight]" duckdb pandas numpy


## 📚 Introdução

Este notebook aborda Arrow Flight SQL:
- Arquitetura Flight SQL
- Servidor e cliente
- Streaming via gRPC
- Prepared statements
- Autenticação

In [2]:
# -*- coding: utf-8 -*-
"""
Capítulo 05: Arrow Flight SQL
Curso: Apache Arrow + DuckDB
"""

import sys
import pyarrow as pa
import pyarrow.flight as flight
try:
    import pyarrow.flight_sql as flight_sql
except ImportError:
    # Flight SQL as a separate module might depend on pyarrow version
    flight_sql = None

import duckdb
import pandas as pd
import numpy as np
import threading
import time

print("="*60)
print(f"CAPÍTULO 05: ARROW FLIGHT SQL")
print("="*60)


CAPÍTULO 05: ARROW FLIGHT SQL


## 🔧 Preparação dos Dados

Criação de dados de exemplo e conexão com DuckDB

In [3]:
# Dados de exemplo globais
try:
    print("\nGerando dados de exemplo...")
    data = pa.table({
        'id': range(1000),
        'valor': np.random.randn(1000),
        'categoria': np.random.choice(['A', 'B', 'C'], 1000)
    })
    print(f"Tabela PyArrow criada: {data.num_rows} linhas")
except Exception as e:
    print(f"Erro ao criar dados: {e}")

# Conexão DuckDB
con = duckdb.connect()


Gerando dados de exemplo...
Tabela PyArrow criada: 1000 linhas


## 🏗️ Tópico 1: Arquitetura Flight SQL

Entendendo a arquitetura e componentes do Arrow Flight SQL

In [4]:
print(f"\n--- {'Arquitetura Flight SQL'.upper()} ---")

# 5.1.1 O que é Arrow Flight?
print("1. Fundamentos do Arrow Flight:")
print("-" * 40)
print("Arrow Flight é um framework RPC otimizado para transferência de dataframes.")
print("Diferente de ODBC/JDBC que serializam dados linha a linha (lento),")
print("o Flight transfere buffers Arrow diretamente sobre gRPC (rápido).")

# 5.1.2 Flight SQL
print("\n2. Flight SQL:")
print("-" * 40)
print("É uma camada sobre o Flight que define comandos SQL padrão.")
print("- Clients podem enviar: Execute, GetTables, GetSchemas.")
print("- Permite interoperabilidade entre bancos de dados sem drivers customizados.")

# 5.1.3 FlightDescriptor
# Criando um descritor de comando SQL simples (apenas demonstração conceitual)
sql_command = "SELECT * FROM sales_data"
descriptor = flight.FlightDescriptor.for_command(sql_command.encode('utf-8'))

print(f"\nExemplo de FlightDescriptor:")
print(f"  Tipo: {descriptor.descriptor_type}")
print(f"  Comando: {descriptor.command.decode('utf-8')}")

# 5.1.4 FlightInfo
# FlightInfo contém os 'endpoints' onde os dados podem ser recuperados
schema = pa.schema([('id', pa.int64()), ('valor', pa.float64())])
locations = [flight.Location.for_grpc_tcp("localhost", 8888)]
endpoints = [flight.FlightEndpoint("ticket-001", locations)]

flight_info = flight.FlightInfo(schema, descriptor, endpoints, total_records=1000, total_bytes=-1)

print(f"\nFlightInfo (Metadados da Query):")
print(f"  Schema: {flight_info.schema}")
print(f"  Endpoints detectados: {len(flight_info.endpoints)}")
print(f"  Total de registros esperados: {flight_info.total_records}")



--- ARQUITETURA FLIGHT SQL ---
1. Fundamentos do Arrow Flight:
----------------------------------------
Arrow Flight é um framework RPC otimizado para transferência de dataframes.
Diferente de ODBC/JDBC que serializam dados linha a linha (lento),
o Flight transfere buffers Arrow diretamente sobre gRPC (rápido).

2. Flight SQL:
----------------------------------------
É uma camada sobre o Flight que define comandos SQL padrão.
- Clients podem enviar: Execute, GetTables, GetSchemas.
- Permite interoperabilidade entre bancos de dados sem drivers customizados.

Exemplo de FlightDescriptor:
  Tipo: DescriptorType.CMD
  Comando: SELECT * FROM sales_data

FlightInfo (Metadados da Query):
  Schema: id: int64
valor: double
  Endpoints detectados: 1
  Total de registros esperados: 1000


## 🔌 Tópico 2: Servidor e cliente

Implementação de servidor e cliente Flight SQL

In [5]:
print(f"\n--- {'Servidor e cliente'.upper()} ---")

# 5.2.1 Criando um Servidor Flight Simples
class SimpleFlightServer(flight.FlightServerBase):
    def __init__(self, location, table_data, **kwargs):
        super(SimpleFlightServer, self).__init__(location, **kwargs)
        self.table_data = table_data
        self.location = location

    def list_flights(self, context, criteria):
        # Descreve os dados disponíveis
        descriptor = flight.FlightDescriptor.for_path("dataset_principal")
        endpoints = [flight.FlightEndpoint("ticket-main", [self.location])]
        return [flight.FlightInfo(self.table_data.schema, descriptor, endpoints, self.table_data.num_rows, -1)]

    def get_flight_info(self, context, descriptor):
        # Retorna info para um descritor específico
        endpoints = [flight.FlightEndpoint("ticket-main", [self.location])]
        return flight.FlightInfo(self.table_data.schema, descriptor, endpoints, self.table_data.num_rows, -1)

    def do_get(self, context, ticket):
        # Envia os dados reais
        return flight.RecordBatchStream(self.table_data)

# 5.2.2 Iniciando o Servidor em Background
# Usamos 127.0.0.1 para evitar problemas com resolução de 'localhost' em alguns sistemas
# e tentamos portas diferentes caso a porta padrão esteja ocupada.
port = 8888
server = None
location_str = f"grpc+tcp://127.0.0.1:{port}"

while port < 8895:
    try:
        location_str = f"grpc+tcp://127.0.0.1:{port}"
        server = SimpleFlightServer(location_str, data)
        print(f"Servidor instanciado com sucesso na porta {port}")
        break
    except Exception as e:
        print(f"Porta {port} ocupada ou erro: {e}. Tentando próxima...")
        port += 1

if server:
    def run_server():
        print(f"Iniciando servidor Flight em {location_str}...")
        server.serve()

    # Rodar em thread para não travar o notebook
    thread = threading.Thread(target=run_server, daemon=True)
    thread.start()
    time.sleep(1) # Aguardar inicialização

    # 5.2.3 Criando o Cliente e Consumindo Dados
    print("\nConectando cliente ao servidor...")
    client = flight.connect(location_str)

    # Listar voos disponíveis
    try:
        flights = list(client.list_flights())
        print(f"Voos encontrados no servidor: {len(flights)}")

        for info in flights:
            print(f"  Consumindo: {info.descriptor.path[0].decode('utf-8')}")
            
            # Solicitar stream de dados usando o ticket
            ticket = info.endpoints[0].ticket
            reader = client.do_get(ticket)
            
            # Converter stream para tabela Arrow
            result_table = reader.read_all()
            print(f"  Tabela recebida: {result_table.num_rows} linhas")
            print(f"  Schema: {result_table.schema.names}")
    except Exception as e:
        print(f"Erro ao consumir dados: {e}")
else:
    print("Não foi possível iniciar o servidor Flight em nenhuma das portas testadas.")


--- SERVIDOR E CLIENTE ---
Servidor instanciado com sucesso na porta 8888
Iniciando servidor Flight em grpc+tcp://127.0.0.1:8888...

Conectando cliente ao servidor...
Voos encontrados no servidor: 1
  Consumindo: dataset_principal
  Tabela recebida: 1000 linhas
  Schema: ['id', 'valor', 'categoria']


## 📡 Tópico 3: Streaming via gRPC

Comunicação eficiente através de streaming gRPC

In [6]:
print(f"\n--- {'Streaming via gRPC'.upper()} ---")

# 5.3.1 Conceito de Streaming no Flight
print("1. Eficiência do Streaming:")
print("-" * 40)
print("No Flight, os dados não são 'baixados' como um arquivo gigante.")
print("Eles fluem como RecordBatches sobre gRPC, permitindo processamento imediato.")

# 5.3.2 Lendo dados em Batches (Chunks)
print("\n2. Leitura Batch-by-Batch:")
descriptor = flight.FlightDescriptor.for_path("dataset_principal")
info = client.get_flight_info(descriptor)
reader = client.do_get(info.endpoints[0].ticket)

print(f"Iniciando consumo de stream...")
total_rows = 0
batch_count = 0

start_time = time.perf_counter()

# Iterar sobre os batches conforme eles chegam do gRPC
for chunk in reader:
    batch_count += 1
    batch = chunk.data
    total_rows += batch.num_rows
    
    if batch_count <= 3:
        print(f"  Batch {batch_count}: Recebidas {batch.num_rows} linhas")

end_time = time.perf_counter()

print(f"\nStream Finalizado:")
print(f"  Total de Batches: {batch_count}")
print(f"  Total de Linhas: {total_rows}")
print(f"  Tempo total: {end_time - start_time:.4f}s")

# 5.3.3 Comparação gRPC vs REST (Conceitual)
print("\n3. Por que gRPC?")
print("-" * 40)
print("- Protocol Buffer (binário) vs JSON (texto)")
print("- HTTP/2 multiplexado")
print("- Zero-Copy nativo no Arrow")



--- STREAMING VIA GRPC ---
1. Eficiência do Streaming:
----------------------------------------
No Flight, os dados não são 'baixados' como um arquivo gigante.
Eles fluem como RecordBatches sobre gRPC, permitindo processamento imediato.

2. Leitura Batch-by-Batch:
Iniciando consumo de stream...
  Batch 1: Recebidas 1000 linhas

Stream Finalizado:
  Total de Batches: 1
  Total de Linhas: 1000
  Tempo total: 0.0004s

3. Por que gRPC?
----------------------------------------
- Protocol Buffer (binário) vs JSON (texto)
- HTTP/2 multiplexado
- Zero-Copy nativo no Arrow


## 📝 Tópico 4: Prepared statements

Otimização com prepared statements

In [7]:
print(f"\n--- {'Prepared statements'.upper()} ---")

# 5.4.1 O que são Prepared Statements no Flight SQL?
print("1. Conceito de Prepared Statement:")
print("-" * 40)
print("Permite que o servidor compile a query uma vez e o cliente a execute múltiplas vezes")
print("com diferentes parâmetros, economizando tempo de parsing no banco de dados.")

# Fluxo Flight SQL (Pseudo-código explicativo):
# 1. client.prepare("SELECT * FROM table WHERE id = ?") -> PreparedStatement object
# 2. stmt.set_parameter(0, 100)
# 3. info = stmt.execute()
# 4. reader = client.do_get(info.endpoints[0].ticket)

# 5.4.2 Demonstração de Parametrização via Flight
print("\n2. Execução Parametrizada (Simulação):")
print("-" * 40)

def execute_query_with_params(client, query_id, params):
    # No Flight puro, passamos parâmetros via comando no descritor
    # O Flight SQL formaliza isso.
    command = {
        "query_id": query_id,
        "parameters": params
    }
    import json
    descriptor = flight.FlightDescriptor.for_command(json.dumps(command).encode('utf-8'))
    
    print(f"Enviando requisição para query '{query_id}' com parâmetros: {params}")
    # info = client.get_flight_info(descriptor)
    # ... segue o fluxo normal do do_get
    print("  Query disparada e plano de execução reutilizado no servidor.")

execute_query_with_params(client, "get_user_by_id", {"id": 123})
execute_query_with_params(client, "get_user_by_id", {"id": 456})

print("\nBenefícios:")
print("- Prevenção de SQL Injection")
print("- Melhora drástica de performance em queries repetitivas")
print("- Menor overhead de rede (enviamos apenas IDs de queries e dados binários)")



--- PREPARED STATEMENTS ---
1. Conceito de Prepared Statement:
----------------------------------------
Permite que o servidor compile a query uma vez e o cliente a execute múltiplas vezes
com diferentes parâmetros, economizando tempo de parsing no banco de dados.

2. Execução Parametrizada (Simulação):
----------------------------------------
Enviando requisição para query 'get_user_by_id' com parâmetros: {'id': 123}
  Query disparada e plano de execução reutilizado no servidor.
Enviando requisição para query 'get_user_by_id' com parâmetros: {'id': 456}
  Query disparada e plano de execução reutilizado no servidor.

Benefícios:
- Prevenção de SQL Injection
- Melhora drástica de performance em queries repetitivas
- Menor overhead de rede (enviamos apenas IDs de queries e dados binários)


## 🔐 Tópico 5: Autenticação

Implementando autenticação segura

In [8]:
print(f"\n--- {'Autenticação'.upper()} ---")

# 5.5.1 Métodos de Autenticação no Flight
print("1. Tipos de Autenticação:")
print("-" * 40)
print("- Basic Auth (User/Password)")
print("- Bearer Token (JWT/OAuth2)")
print("- TLS/mTLS (Certificados)")

# 5.5.2 Exemplo: Cliente com Token JWT
print("\n2. Exemplo de Cliente Autenticado (CallOptions):")
print("-" * 40)

# Simular um token JWT
auth_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9..."

# Call options permitem injetar headers em cada chamada
options = flight.FlightCallOptions(headers=[
    (b"authorization", f"Bearer {auth_token}".encode('utf-8'))
])

print(f"Configurando headers de autorização...")

try:
    # Tentando listar voos com o header de autenticação
    # Nota: Nosso servidor de teste SimpleFlightServer não valida o token,
    # mas o cliente está enviando corretamente.
    flights = list(client.list_flights(options=options))
    print(f"Chamada autenticada enviada com sucesso.")
    print(f"Token enviado no header 'authorization'")
except Exception as e:
    print(f"Erro na autenticação: {e}")

# 5.5.3 Middleware do Servidor (Conceitual)
print("\n3. Validação no Servidor:")
print("-" * 40)
print("No servidor, implementamos um 'FlightServerMiddleware' para:")
print("1. Interceptar a requisição gRPC")
print("2. Ler os headers (Extract headers)")
print("3. Validar o token ou credenciais")
print("4. Rejeitar com UNAUTHORIZED se inválido")



--- AUTENTICAÇÃO ---
1. Tipos de Autenticação:
----------------------------------------
- Basic Auth (User/Password)
- Bearer Token (JWT/OAuth2)
- TLS/mTLS (Certificados)

2. Exemplo de Cliente Autenticado (CallOptions):
----------------------------------------
Configurando headers de autorização...
Chamada autenticada enviada com sucesso.
Token enviado no header 'authorization'

3. Validação no Servidor:
----------------------------------------
No servidor, implementamos um 'FlightServerMiddleware' para:
1. Interceptar a requisição gRPC
2. Ler os headers (Extract headers)
3. Validar o token ou credenciais
4. Rejeitar com UNAUTHORIZED se inválido
